<a href="https://colab.research.google.com/github/nitthapr/NER_german_lrec2014/blob/master/Try_03_1_BERT_Thai_NER_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Version Note <br>
Try use with preprocess.py

In [2]:
!pip3 install git+https://github.com/huggingface/transformers.git --upgrade

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-vue97eh4
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-vue97eh4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-3.2.0-cp36-none-any.whl size=1058986 sha256=60aace21ae3d5bcd2eb62a1f43e931dcfb29b68b3c77093bf409a24d51dadb7c
  Stored in directory: /tmp/pip-ephem-wheel-cache-20wgt7j6/wheels/33/eb/3b/4bf5dd835e865e472d4fc0754f35ac0edb08fe852e8f21655f
Successfully built transformers
  Found existing installation: transformers 3.2.0
    Uninstalling transformers-3.2.0:
      Successfully uninstalled transformers-3.2.0


In [1]:
!wget -O preprocess_thai.py 'https://raw.githubusercontent.com/nitthapr/NER_german_lrec2014/master/preprocess_thai.py'
!wget -O utils_ner.py 'https://raw.githubusercontent.com/nitthapr/NER_german_lrec2014/master/utils_ner.py'
!wget -O tasks.py 'https://raw.githubusercontent.com/huggingface/transformers/master/examples/token-classification/tasks.py'

--2020-09-27 10:54:46--  https://raw.githubusercontent.com/nitthapr/NER_german_lrec2014/master/preprocess_thai.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1439 (1.4K) [text/plain]
Saving to: ‘preprocess_thai.py’

preprocess_thai.py  100%[===================>]   1.41K  --.-KB/s    in 0s      

2020-09-27 10:54:46 (21.2 MB/s) - ‘preprocess_thai.py’ saved [1439/1439]

--2020-09-27 10:54:46--  https://raw.githubusercontent.com/nitthapr/NER_german_lrec2014/master/utils_ner.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15962 (16K) [te

## 2) Load data

### 2.1) Load pythai data



In [4]:
from sklearn.model_selection import train_test_split

In [5]:
PROJECT_DATA_PATH = '/content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER'

pythai_dataset_path = f'{PROJECT_DATA_PATH}/in/train.data'
ner_dataset_path = f'{PROJECT_DATA_PATH}/in/ner_dataset_pythai.csv'
output_model_path = f'{PROJECT_DATA_PATH}/model/'
checkpoint_path = f'{PROJECT_DATA_PATH}/checkpoint/'
bert_thai_model_path = f'{PROJECT_DATA_PATH}/model/bert_base_th/bert_base_th'

In [6]:
import dill
with open(pythai_dataset_path, 'rb') as file:
 datatofile = dill.load(file)

In [7]:
tagged_sents = []
for i in datatofile:
  text_inside = []
  for j in i:
    text_inside.append((j[0],j[2]))
  tagged_sents.append(text_inside)

train_sents, test_sents = train_test_split(tagged_sents, test_size=0.2, random_state=112)

### 2.2) Turn to conll-2003  format


In [9]:
def turn_pyner_to_colln (data_in, out_filename):
  # For each sentence
  #     1) if word is " " -> replace with "_"
  #     2) insert " " to separate word and label
  #     3) insert label 
  #     4) after sentence end insert blank line 

  # For each sentence
  outtxt = ''
  for sent in data_in:
    outlines = ''
    for word_pair in sent:
      word = word_pair[0]
      label = word_pair[1]
  #     1) if word is " " -> replace with "_"
      if (word == ' '):
        outlines += '_'
      if (' ' in word):
        outlines += word.replace(' ', '_')
      else:
        outlines += word
  #     2) insert " " to separate word and label
      outlines += ' '
  #     3) insert label , go new line for next word
      outlines += label + '\n'
  # 4) after sentence end insert blank line 
    outtxt = outtxt + outlines + '\n'

  with open(out_filename, 'w') as out_file:
     out_file.write(outtxt)

# Test
# data_in = train_sents[1:5]
# p_out_filename = 'pyner_train_sents.txt'
# turn_pyner_to_colln(data_in, p_out_filename)

In [10]:
train_sents, test_sents 
turn_pyner_to_colln(train_sents,'pytrain.txt.tmp')
turn_pyner_to_colln(test_sents,'pytest.txt.tmp')

## 3.) Load ThaiTokenizer


In [11]:
import sys

# from transformers import AutoTokenizer

dataset = 'pytest.txt.tmp' # sys.argv[1]
dataoutput = 'pytest.txt'
model_name_or_path = 'Not use' # sys.argv[2]
max_len = 128 # int(sys.argv[3])

subword_len_counter = 0

In [12]:
# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, f'{PROJECT_DATA_PATH}/model/bert-master')
import tokenization

bpe_vocab_path = f'{PROJECT_DATA_PATH}/model/th_wiki_bpe/th.wiki.bpe.op25000.vocab'
bpe_model_path = f'{PROJECT_DATA_PATH}/model/th_wiki_bpe/th.wiki.bpe.op25000.model'
tokenizer = tokenization.ThaiTokenizer(vocab_file=bpe_vocab_path, spm_file=bpe_model_path)

In [13]:
sys.path

['',
 '/content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/model/bert-master',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [14]:
# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
# max_len -= tokenizer.num_special_tokens_to_add()
max_len -= 2

### Final Thai preprocess

In [15]:
import sys

# from transformers import AutoTokenizer

# dataset = # sys.argv[1]
model_name_or_path = 'Not use' # sys.argv[2]
max_len = 128 # int(sys.argv[3])
PROJECT_DATA_PATH = '/content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER'

subword_len_counter = 0

# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, f'{PROJECT_DATA_PATH}/model/bert-master')
import tokenization

bpe_vocab_path = f'{PROJECT_DATA_PATH}/model/th_wiki_bpe/th.wiki.bpe.op25000.vocab'
bpe_model_path = f'{PROJECT_DATA_PATH}/model/th_wiki_bpe/th.wiki.bpe.op25000.model'
tokenizer = tokenization.ThaiTokenizer(vocab_file=bpe_vocab_path, spm_file=bpe_model_path)
max_len -= 2  # my set , must recheck

with open(dataset, "rt") as f_p:
    for line in f_p:
        line = line.rstrip()

        if not line:
            print(line)
            subword_len_counter = 0
            continue

        token = line.split()[0]

        current_subwords_len = len(tokenizer.tokenize(token))

        # Token contains strange control characters like \x96 or \x95
        # Just filter out the complete line
        if current_subwords_len == 0:
            continue

        if (subword_len_counter + current_subwords_len) > max_len:
            print("")
            print(line)
            subword_len_counter = current_subwords_len
            continue

        subword_len_counter += current_subwords_len

        print(line)

Streaming output truncated to the last 5000 lines.
คึกคัก O
__ O
โดยเฉพาะ O
แหล่ง O
ซื้อขาย O
ดอกไม้ O
__ O
พบ O
ว่า O
ทุ O
กร้าน O
ต่าง O
จัด O
เต็ม O
__ O
ทั้ง O
ตกแต่ง O
ร้าน O
ให้ O
เข้ากับ O
บรรยากาศ O
__ O
และ O
ตั้ง O
วางขาย O
เพื่อ O
ดึงดูด O
ลูกค้า O
โดยเฉพาะ O
เด็กนักเรียน O
__ O
เข้ามา O
เลือก O
ซื้อ O
ไป O
แสดง O
ความรัก O
แก่ O
กัน O

สธ B-ORGANIZATION
. I-ORGANIZATION
__ O
ยก O
แฝด O
น้อง O
__ O
ให้ O
__ O
ฮู B-ORGANIZATION
__ O
เป็น O
กรณีศึกษา O
หวัด O
นก O
แพร่ O
สู่ O
คน O

ขณะเดียวกัน O
ทีม O
นักวิชาการ O
ผู้เชี่ยวชาญ O
จาก O
ศูนย์ B-ORGANIZATION
วิจัย I-ORGANIZATION
ทรัพยากร I-ORGANIZATION
ทางทะเล I-ORGANIZATION
และ I-ORGANIZATION
ชายฝั่ง I-ORGANIZATION
__ I-ORGANIZATION
อ่าวไทย I-ORGANIZATION
ตอนกลาง I-ORGANIZATION
__ O
จังหวัด B-LOCATION
ชุมพร I-LOCATION
__ O
เข้า O
สำรวจ O
เพื่อ O
วาง O
มาตรการป้องกัน O
ความเสียหาย O
ของ O
แนวปะการัง O
น้ำ O
ตื้น O
ตั้งแต่ O
กลางเดือน B-DATE
มิถุนายน I-DATE
ที่ผ่านมา O
__ O
เบื้องต้น O
ยัง O
ไม่ O
พบ O
ความเสียหาย O
กับ O
ปะการัง

In [16]:
!export MAX_LENGTH=128
!export BERT_MODEL=none

In [17]:
# there is problem with sending param, don't know how to fix yet
!python3 preprocess_thai.py pytrain.txt.tmp none 128 > train.txt
!python3 preprocess_thai.py pytest.txt.tmp none 128 > test.txt

2020-09-27 10:58:01.183800: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-27 10:58:10.029333: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [18]:
# *** Dont' have to recreate , if recreate must re insert PAD
# !cat train.txt test.txt  | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt

# 4.) Run run_ner


In [19]:
!pip3  install seqeval
!pip3 install conllu

In [20]:
import logging
import os
import sys
from dataclasses import dataclass, field
from importlib import import_module
from typing import Dict, List, Optional, Tuple

import numpy as np
from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score
from torch import nn

from transformers import (
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
)


from utils_ner import Split, TokenClassificationDataset, TokenClassificationTask
logger = logging.getLogger(__name__)


In [21]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    task_type: Optional[str] = field(
        default="NER", metadata={"help": "Task type to fine tune in training (e.g. NER, POS, etc)"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    use_fast: bool = field(default=False, metadata={"help": "Set this flag to use fast tokenization."})
    # If you want to tweak more attributes on your tokenizer, you should do it in a distinct script,
    # or just modify its tokenizer_config.json.
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    data_dir: str = field(
        metadata={"help": "The input data dir. Should contain the .txt files for a CoNLL-2003-formatted task."}
    )
    labels: Optional[str] = field(
        default=None,
        metadata={"help": "Path to a file containing all labels. If not specified, CoNLL-2003 labels are used."},
    )
    max_seq_length: int = field(
        default=128,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )


In [22]:
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))

In [49]:
# p_config = f'{PROJECT_DATA_PATH}/output/test_run_ner2_TH/run_ner_config.json'
p_config = f'{PROJECT_DATA_PATH}/output/test_run_ner2_TH/run_ner_config_multilm.json'
p_config2 = f'{PROJECT_DATA_PATH}/output/test_run_ner2_TH/run_ner_config_multilm.json'

In [50]:
if True:
### Test run with json file
# if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
    # If we pass only one argument to the script and it's the path to a json file,
    # let's parse it to get our arguments.
    model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath(p_config2))
else:
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()

if (
    # if already exists output directory -> raise Error
    os.path.exists(training_args.output_dir)
    and os.listdir(training_args.output_dir)
    and training_args.do_train
    and not training_args.overwrite_output_dir
):  
    raise ValueError(
        f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
    )

In [51]:
module = import_module("tasks")
try:### ??? DO I have to change? 
    token_classification_task_clazz = getattr(module, model_args.task_type)
    token_classification_task: TokenClassificationTask = token_classification_task_clazz()
except AttributeError:
    raise ValueError(
        f"Task {model_args.task_type} needs to be defined as a TokenClassificationTask subclass in {module}. "
        f"Available tasks classes are: {TokenClassificationTask.__subclasses__()}"
    )

# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
)
logger.warning(
    "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
    training_args.local_rank,
    training_args.device,
    training_args.n_gpu,
    bool(training_args.local_rank != -1),
    training_args.fp16,
)
logger.info("Training/evaluation parameters %s", training_args)

# Set seed
set_seed(training_args.seed)

09/27/2020 11:12:33 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
09/27/2020 11:12:33 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='bert_fine_TH', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=None, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3, max_steps=-1, warmup_steps=0, logging_dir='runs/Sep27_11-12-29_2dbb8df8eb37', logging_first_step=False, logging_steps=500, save_steps=750, save_total_limit=None, no_cuda=False, seed=1, fp16=False, fp16_opt_level='O1', local_rank=-1, tpu_num_cores=None, tpu_metrics_debug=F

### 4.1) Prep data

In [52]:
# Prepare CONLL-2003 task
### --MUST CHANGE---
labels = token_classification_task.get_labels(data_args.labels)
label_map: Dict[int, str] = {i: label for i, label in enumerate(labels)}
num_labels = len(label_map)

In [53]:
label_map

{0: 'PAD',
 1: 'B-ไม่ยืนยัน',
 2: 'B-DATE',
 3: 'B-EMAIL',
 4: 'B-LAW',
 5: 'B-LEN',
 6: 'B-LOCATION',
 7: 'B-MONEY',
 8: 'B-ORGANIZATION',
 9: 'B-PERCENT',
 10: 'B-PERSON',
 11: 'B-PHONE',
 12: 'B-TIME',
 13: 'B-URL',
 14: 'B-ZIP',
 15: 'I-ไม่ยืนยัน',
 16: 'I-DATE',
 17: 'I-EMAIL',
 18: 'I-LAW',
 19: 'I-LEN',
 20: 'I-LOCATION',
 21: 'I-MONEY',
 22: 'I-ORGANIZATION',
 23: 'I-PERCENT',
 24: 'I-PERSON',
 25: 'I-PHONE',
 26: 'I-TIME',
 27: 'I-URL',
 28: 'O'}

In [54]:
from transformers import BertForTokenClassification, AdamW, BertModel, BertConfig

In [55]:
# -- Just for test
import os.path
os.path.isfile('/content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/model/bert_base_th/bert_base_th/model.ckpt')

False

In [30]:
path_lm_config = (f'{bert_thai_model_path}/bert_config.json')

In [31]:
print(model_args.model_name_or_path)
if model_args.config_name :
  print('Yesss')
print(model_args.config_name)
bool(bool(".ckpt" in model_args.model_name_or_path))

/content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/model/bert_base_th/bert_base_th/model.ckpt.index
None


True

In [56]:
config = AutoConfig.from_pretrained(
# --- olunlah
#    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    path_lm_config ,
    num_labels=num_labels,
    id2label=label_map,
    label2id={label: i for i, label in enumerate(labels)}
    #cache_dir=model_args.cache_dir,
)
tokenizer =  tokenization.ThaiTokenizer(vocab_file=bpe_vocab_path, spm_file=bpe_model_path)

In [44]:
model = AutoModelForTokenClassification.from_pretrained(
        model_args.model_name_or_path,
        from_tf=bool(".ckpt" in model_args.model_name_or_path),
        config=config
        #cache_dir=model_args.cache_dir,
    )

OSError: ignored

In [ ]:
# Load pretrained model and tokenizer
#
# Distributed training:
# The .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.

### --MUST CHANGE---
config = AutoConfig.from_pretrained(
# --- olunlah
#    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    path_lm_config ,
    num_labels=num_labels,
    id2label=label_map,
    label2id={label: i for i, label in enumerate(labels)}
    #cache_dir=model_args.cache_dir,
)
#tokenizer = AutoTokenizer.from_pretrained(
#    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
#    cache_dir=model_args.cache_dir,
#    use_fast=model_args.use_fast,
#)

# config = BertConfig.from_json_file(f'{bert_thai_model_path}/bert_config.json')


model = AutoModelForTokenClassification.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config
    #cache_dir=model_args.cache_dir,
)

# old # model = AutoModelForTokenClassification.from_config(config)

In [57]:
config = AutoConfig.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    num_labels=num_labels,
    id2label=label_map,
    label2id={label: i for i, label in enumerate(labels)},
    cache_dir=model_args.cache_dir,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
    use_fast=model_args.use_fast,
)
model = AutoModelForTokenClassification.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
    cache_dir=model_args.cache_dir,
)


09/27/2020 11:12:57 - INFO - filelock -   Lock 139746816062632 acquired on /root/.cache/torch/transformers/45629519f3117b89d89fd9c740073d8e4c1f0a70f9842476185100a8afe715d1.65df3cef028a0c91a7b059e4c404a975ebe6843c71267b67019c0e9cfa8a88f0.lock


09/27/2020 11:12:57 - INFO - filelock -   Lock 139746816062632 released on /root/.cache/torch/transformers/45629519f3117b89d89fd9c740073d8e4c1f0a70f9842476185100a8afe715d1.65df3cef028a0c91a7b059e4c404a975ebe6843c71267b67019c0e9cfa8a88f0.lock


09/27/2020 11:12:57 - INFO - filelock -   Lock 139746771421112 acquired on /root/.cache/torch/transformers/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729.lock


09/27/2020 11:12:58 - INFO - filelock -   Lock 139746771421112 released on /root/.cache/torch/transformers/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729.lock


09/27/2020 11:12:58 - INFO - filelock -   Lock 139746769521968 acquired on /root/.cache/torch/transformers/3d1d2b2daef1e2b3ddc2180ddaae8b7a37d5f279babce0068361f71cd548f615.7131dcb754361639a7d5526985f880879c9bfd144b65a0bf50590bddb7de9059.lock


09/27/2020 11:13:54 - INFO - filelock -   Lock 139746769521968 released on /root/.cache/torch/transformers/3d1d2b2daef1e2b3ddc2180ddaae8b7a37d5f279babce0068361f71cd548f615.7131dcb754361639a7d5526985f880879c9bfd144b65a0bf50590bddb7de9059.lock


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

In [ ]:
#config = BertConfig.from_json_file(f'{bert_thai_model_path}/bert_config.json')
#tokenizer =  tokenization.ThaiTokenizer(vocab_file=bpe_vocab_path, spm_file=bpe_model_path)
#model = AutoModelForTokenClassification.from_pretrained(model_args.model_name_or_path, from_tf=True, config=config)

In [58]:
from utils_ner import InputExample, Split, TokenClassificationTask
data_dir = './'
mode=Split.train
if isinstance(mode, Split):
  mode = mode.value
file_path = os.path.join(data_dir, f"{mode}.txt")
file_path

print(file_path)

./train.txt


In [59]:
mode_tr=Split.train
type(mode_tr)
mode_ev=Split.test
type(mode_ev)

<enum 'Split'>

In [60]:
# Get datasets
train_dataset = (
    TokenClassificationDataset(
        token_classification_task=token_classification_task,
        data_dir=data_args.data_dir,
        tokenizer=tokenizer,
        labels=labels,
        model_type=config.model_type,
        max_seq_length=data_args.max_seq_length,
        overwrite_cache=data_args.overwrite_cache,
        mode=mode_tr
        #mode=Split.train
    )
    # if training_args.do_train
    if True
    else None
)
eval_dataset = (
    TokenClassificationDataset(
        token_classification_task=token_classification_task,
        data_dir=data_args.data_dir,
        tokenizer=tokenizer,
        labels=labels,
        model_type=config.model_type,
        max_seq_length=data_args.max_seq_length,
        overwrite_cache=data_args.overwrite_cache,
        #mode=Split.dev,
        mode=Split.test
    )
    #if training_args.do_eval
    if True
    else None
)

09/27/2020 11:14:26 - INFO - filelock -   Lock 139746816325224 acquired on ./cached_train_BertTokenizer_128.lock
09/27/2020 11:14:26 - INFO - utils_ner -   Creating features from dataset file at .
09/27/2020 11:14:27 - INFO - utils_ner -   Writing example 0 of 5568
09/27/2020 11:14:27 - INFO - utils_ner -   *** Example ***
09/27/2020 11:14:27 - INFO - utils_ner -   guid: train-1
09/27/2020 11:14:27 - INFO - utils_ner -   tokens: [CLS] [UNK] ห ##ัว เ ##ว ##่ ##ย [UNK] _ _ ข ##ึ ##้น ##แ ##ท ##่น เ ##บ ##อร์ _ _ 2 _ _ ส ##มา ##ร์ ##ท โ ##ฟ ##น โ ##ลก _ _ โ ##ช ##ว ##์ ส ##่ ##วน ##แ ##บ ##่ง ##ต ##ลา ##ด _ _ 15 . 4 % [SEP]
09/27/2020 11:14:27 - INFO - utils_ner -   input_ids: 101 100 1434 65416 1450 31287 31904 20503 100 168 168 1396 111428 49512 111432 37022 34523 1450 49097 37760 168 168 123 168 168 1433 66845 68458 37022 1452 72245 16000 1452 102300 168 168 1452 42407 31287 26832 1433 31904 73179 111432 49097 44334 30011 61888 22123 168 168 10208 119 125 110 102 0 0 0 0 0 0 0 0 0 0 0 

AssertionError: ignored

### 4.2) Other def and run_ner.py





In [ ]:
def align_predictions(predictions: np.ndarray, label_ids: np.ndarray) -> Tuple[List[int], List[int]]:
    preds = np.argmax(predictions, axis=2)

    batch_size, seq_len = preds.shape

    out_label_list = [[] for _ in range(batch_size)]
    preds_list = [[] for _ in range(batch_size)]

    for i in range(batch_size):
        for j in range(seq_len):
            if label_ids[i, j] != nn.CrossEntropyLoss().ignore_index:
                out_label_list[i].append(label_map[label_ids[i][j]])
                preds_list[i].append(label_map[preds[i][j]])

    return preds_list, out_label_list

def compute_metrics(p: EvalPrediction) -> Dict:
    preds_list, out_label_list = align_predictions(p.predictions, p.label_ids)
    return {
        "accuracy_score": accuracy_score(out_label_list, preds_list),
        "precision": precision_score(out_label_list, preds_list),
        "recall": recall_score(out_label_list, preds_list),
        "f1": f1_score(out_label_list, preds_list),
    }

In [ ]:
# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# Training
if training_args.do_train:
    trainer.train(
        model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
    )
    trainer.save_model()
    # For convenience, we also re-save the tokenizer to the same directory,
    # so that you can share your model easily on huggingface.co/models =)
    if trainer.is_world_master():
        tokenizer.save_pretrained(training_args.output_dir)


# Checking Code


In [ ]:
label_map

In [ ]:
token_classification_task

In [ ]:

dataset_iter = iter(train_dataset)
for indices in dataset_iter:
  yield collate_fn([next(dataset_iter) for _ in indices])

In [ ]:
train_dataset.__getitem__(2)